In [ ]:
!pip install -q espnet==0.10.3 pyopenjtalk==0.1.5 parallel_wavegan==0.5.3 espnet_model_zoo   #Install the espnet2 model

     |████████████████████████████████| 907 kB 7.4 MB/s 
     |████████████████████████████████| 1.5 MB 55.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 68 kB 6.8 MB/s 
     |████████████████████████████████| 212 kB 71.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 125 kB 57.5 MB/s 
     |████████████████████████████████| 1.8 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 174 kB 72.3 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 1.2 MB 59.1 MB/s 
     |████████████████████████████████| 71 kB 10.4 MB/s 
     |████████████████████████████████| 13.1 MB 25.9 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
  

In [ ]:
from espnet2.bin.tts_inference import Text2Speech   # import Text2Speech from espnet2
from espnet2.utils.types import str_or_none
import time
import torch
import tracemalloc

tracemalloc.start()
begin = time.time()
lang = 'English'  # Other supported languages: Japanese, Chinese, Mandarin and many others.

# Using the pre-trained VITs LJSpeech model. This can be easily replaced with the options shown below:
tag = 'kan-bayashi/ljspeech_vits'

""" Any of the acoustic models listed below can be used in place of the tag:
kan-bayashi/ljspeech_fastspeech, kan-bayashi/ljspeech_tacotron2, kan-bayashi/ljspeech_vits, 
kan-bayashi/ljspeech_fastspeech2, kan-bayashi/ljspeech_conformer_fastspeech2
"""

"""We are not using a vocoder because VITS does not require one. You can still use one and experiment despite that with the ones listed below."""
vocoder_tag = "none"

""" The following vocoders can be used instead of the above one:
parallel_wavegan/ljspeech_parallel_wavegan.v1, parallel_wavegan/ljspeech_style_melgan.v1, 
parallel_wavegan/ljspeech_full_band_melgan.v2, parallel_wavegan/ljspeech_multi_band_melgan.v2, parallel_wavegan/ljspeech_hifigan.v1  """

text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    vocoder_tag=str_or_none(vocoder_tag),
    device="cuda",
    
    # For Tacotron 2 & Transformer only
    threshold=0.5,
    
    # For Tacotron 2 only
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    
    speed_control_alpha=1.0,  # For FastSpeech & FastSpeech2 & VITS only
    
    # For VITS only
    noise_scale=0.333,
    noise_scale_dur=0.333,
)

input_text = 'ERROR: pips dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.'

# Start the speech synthesis:
with torch.no_grad():
    start = time.time()
    wav = text2speech(input_text)["wav"]
rtf = (time.time() - start) / (len(wav) / text2speech.fs)
print(f"RTF = {rtf:5f}")   # Although there isn't a definite limit to the amount of text you may input, a large text block will gradually lower your RTF score. Work should be done in larger sections.

from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))

terminate = time.time()
print("Time consumed: ", terminate-begin)
print("Memory consumed: ", tracemalloc.get_traced_memory())
tracemalloc.stop()

RTF = 0.134912


Time consumed:  5.3493266105651855
Memory consumed:  (69160443, 108808061)
